In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

#### TODO
1. waht is the usage of six

In [2]:
pickle_file = "/home/big_muscle/data/notMNIST.pickle"

In [3]:
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (500000, 28, 28) (500000,)
Validation set (20000, 28, 28) (20000,)
Test set (10000, 28, 28) (10000,)


1. open the pickle file and load the data.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (500000, 28, 28, 1) (500000, 10)
Validation set (20000, 28, 28, 1) (20000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


1. use the on-hot to reshape labels.
why 2. add the channel number in the data set.

In [5]:
def accuracy(predictions, labels):
    """
    all the right predictions in the data
    """
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool_layer = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool_layer, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.779096
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.980206
Minibatch accuracy: 18.8%
Validation accuracy: 34.7%
Minibatch loss at step 100: 1.268485
Minibatch accuracy: 56.2%
Validation accuracy: 61.4%
Minibatch loss at step 150: 0.848286
Minibatch accuracy: 75.0%
Validation accuracy: 72.7%
Minibatch loss at step 200: 0.632075
Minibatch accuracy: 87.5%
Validation accuracy: 78.4%
Minibatch loss at step 250: 1.009703
Minibatch accuracy: 62.5%
Validation accuracy: 73.0%
Minibatch loss at step 300: 0.465876
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 350: 0.381084
Minibatch accuracy: 87.5%
Validation accuracy: 79.4%
Minibatch loss at step 400: 0.667109
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 450: 1.162019
Minibatch accuracy: 56.2%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.597743
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Mi

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

Problem 2
Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

In [23]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data, drop_rate=0.99):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer1_biases), drop_rate)
        pool_layer = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool_layer, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer2_biases), drop_rate)
        hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

steps:30001, dropout:0.99, accuracy:95.7% overfit

In [31]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data, drop_rate=0.99):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer1_biases), drop_rate)
        pool_layer = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool_layer, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer2_biases), drop_rate)
        hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.961884
Minibatch accuracy: 25.0%
Validation accuracy: 10.5%
Minibatch loss at step 50: 1.869950
Minibatch accuracy: 31.2%
Validation accuracy: 47.0%
Minibatch loss at step 100: 0.904656
Minibatch accuracy: 75.0%
Validation accuracy: 72.1%
Minibatch loss at step 150: 0.706282
Minibatch accuracy: 62.5%
Validation accuracy: 77.4%
Minibatch loss at step 200: 0.544970
Minibatch accuracy: 87.5%
Validation accuracy: 77.4%
Minibatch loss at step 250: 0.887972
Minibatch accuracy: 68.8%
Validation accuracy: 79.4%
Minibatch loss at step 300: 0.647771
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 350: 0.375175
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Minibatch loss at step 400: 0.781692
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.935832
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 500: 0.451279
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
M

Validation accuracy: 88.2%
Minibatch loss at step 4550: 0.042707
Minibatch accuracy: 100.0%
Validation accuracy: 88.2%
Minibatch loss at step 4600: 0.310992
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 4650: 0.597843
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Minibatch loss at step 4700: 0.719188
Minibatch accuracy: 81.2%
Validation accuracy: 88.7%
Minibatch loss at step 4750: 0.312227
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 4800: 0.401914
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 4850: 0.487606
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 4900: 0.534248
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 4950: 0.016355
Minibatch accuracy: 100.0%
Validation accuracy: 88.4%
Minibatch loss at step 5000: 0.458191
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 5050: 0.456038
Minibatch accuracy: 81.

Validation accuracy: 89.4%
Minibatch loss at step 9050: 0.506350
Minibatch accuracy: 81.2%
Validation accuracy: 89.5%
Minibatch loss at step 9100: 0.227093
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 9150: 0.437320
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 9200: 0.197952
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 9250: 0.139629
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 9300: 0.245885
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 9350: 0.229480
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 9400: 0.372784
Minibatch accuracy: 87.5%
Validation accuracy: 89.5%
Minibatch loss at step 9450: 0.679061
Minibatch accuracy: 81.2%
Validation accuracy: 89.7%
Minibatch loss at step 9500: 0.853469
Minibatch accuracy: 87.5%
Validation accuracy: 89.5%
Minibatch loss at step 9550: 0.488975
Minibatch accuracy: 81.2%

Validation accuracy: 89.4%
Minibatch loss at step 13550: 0.084005
Minibatch accuracy: 100.0%
Validation accuracy: 89.9%
Minibatch loss at step 13600: 0.584116
Minibatch accuracy: 81.2%
Validation accuracy: 90.0%
Minibatch loss at step 13650: 0.045455
Minibatch accuracy: 100.0%
Validation accuracy: 90.0%
Minibatch loss at step 13700: 0.149042
Minibatch accuracy: 100.0%
Validation accuracy: 89.8%
Minibatch loss at step 13750: 0.065515
Minibatch accuracy: 100.0%
Validation accuracy: 89.8%
Minibatch loss at step 13800: 0.338170
Minibatch accuracy: 87.5%
Validation accuracy: 89.9%
Minibatch loss at step 13850: 0.144767
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 13900: 0.691205
Minibatch accuracy: 75.0%
Validation accuracy: 89.8%
Minibatch loss at step 13950: 0.192924
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 14000: 0.584818
Minibatch accuracy: 75.0%
Validation accuracy: 90.0%
Minibatch loss at step 14050: 0.004780
Minibatch 

Validation accuracy: 90.3%
Minibatch loss at step 18000: 0.413722
Minibatch accuracy: 87.5%
Validation accuracy: 90.1%
Minibatch loss at step 18050: 0.151570
Minibatch accuracy: 93.8%
Validation accuracy: 90.0%
Minibatch loss at step 18100: 0.123302
Minibatch accuracy: 100.0%
Validation accuracy: 90.1%
Minibatch loss at step 18150: 0.483924
Minibatch accuracy: 87.5%
Validation accuracy: 90.1%
Minibatch loss at step 18200: 0.399799
Minibatch accuracy: 81.2%
Validation accuracy: 89.4%
Minibatch loss at step 18250: 0.128505
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 18300: 0.270147
Minibatch accuracy: 87.5%
Validation accuracy: 90.3%
Minibatch loss at step 18350: 0.054423
Minibatch accuracy: 100.0%
Validation accuracy: 90.3%
Minibatch loss at step 18400: 0.121975
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 18450: 0.195059
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 18500: 0.119294
Minibatch ac

Validation accuracy: 90.3%
Minibatch loss at step 22500: 0.287615
Minibatch accuracy: 93.8%
Validation accuracy: 90.4%
Minibatch loss at step 22550: 0.436987
Minibatch accuracy: 81.2%
Validation accuracy: 90.4%
Minibatch loss at step 22600: 0.224054
Minibatch accuracy: 100.0%
Validation accuracy: 90.7%
Minibatch loss at step 22650: 0.104797
Minibatch accuracy: 100.0%
Validation accuracy: 90.5%
Minibatch loss at step 22700: 0.189342
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 22750: 0.015284
Minibatch accuracy: 100.0%
Validation accuracy: 90.5%
Minibatch loss at step 22800: 0.173753
Minibatch accuracy: 93.8%
Validation accuracy: 90.6%
Minibatch loss at step 22850: 0.018975
Minibatch accuracy: 100.0%
Validation accuracy: 90.2%
Minibatch loss at step 22900: 0.188822
Minibatch accuracy: 100.0%
Validation accuracy: 90.3%
Minibatch loss at step 22950: 0.391488
Minibatch accuracy: 93.8%
Validation accuracy: 90.6%
Minibatch loss at step 23000: 0.438054
Minibatch

Minibatch loss at step 26950: 0.494562
Minibatch accuracy: 87.5%
Validation accuracy: 90.6%
Minibatch loss at step 27000: 0.386336
Minibatch accuracy: 87.5%
Validation accuracy: 90.7%
Minibatch loss at step 27050: 0.219110
Minibatch accuracy: 93.8%
Validation accuracy: 90.9%
Minibatch loss at step 27100: 0.184516
Minibatch accuracy: 93.8%
Validation accuracy: 90.6%
Minibatch loss at step 27150: 0.010812
Minibatch accuracy: 100.0%
Validation accuracy: 90.9%
Minibatch loss at step 27200: 0.378817
Minibatch accuracy: 87.5%
Validation accuracy: 90.6%
Minibatch loss at step 27250: 0.002865
Minibatch accuracy: 100.0%
Validation accuracy: 90.8%
Minibatch loss at step 27300: 0.282077
Minibatch accuracy: 87.5%
Validation accuracy: 90.6%
Minibatch loss at step 27350: 0.754035
Minibatch accuracy: 81.2%
Validation accuracy: 90.8%
Minibatch loss at step 27400: 0.778496
Minibatch accuracy: 62.5%
Validation accuracy: 90.6%
Minibatch loss at step 27450: 0.165245
Minibatch accuracy: 93.8%
Validation ac

Adamoptimizer, 30001 iterations, accuracy 95.9, overfit

In [36]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 256

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data, drop_rate=0.99):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer1_biases), drop_rate)
        pool_layer = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool_layer, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(conv + layer2_biases), drop_rate)
        hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.119781
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.817382
Minibatch accuracy: 37.5%
Validation accuracy: 45.2%
Minibatch loss at step 100: 0.701398
Minibatch accuracy: 81.2%
Validation accuracy: 70.7%
Minibatch loss at step 150: 0.805676
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 200: 0.538366
Minibatch accuracy: 81.2%
Validation accuracy: 73.9%
Minibatch loss at step 250: 0.743535
Minibatch accuracy: 68.8%
Validation accuracy: 79.5%
Minibatch loss at step 300: 0.616125
Minibatch accuracy: 75.0%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.412794
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.873469
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 450: 1.226793
Minibatch accuracy: 62.5%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.327470
Minibatch accuracy: 93.8%
Validation accuracy: 83.0%
Mi

Validation accuracy: 88.6%
Minibatch loss at step 4600: 0.288253
Minibatch accuracy: 93.8%
Validation accuracy: 88.3%
Minibatch loss at step 4650: 0.732370
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 4700: 0.818283
Minibatch accuracy: 68.8%
Validation accuracy: 88.7%
Minibatch loss at step 4750: 0.315506
Minibatch accuracy: 100.0%
Validation accuracy: 88.6%
Minibatch loss at step 4800: 0.335193
Minibatch accuracy: 93.8%
Validation accuracy: 88.2%
Minibatch loss at step 4850: 0.625566
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 4900: 0.471776
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 4950: 0.027614
Minibatch accuracy: 100.0%
Validation accuracy: 88.7%
Minibatch loss at step 5000: 0.406566
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 5050: 0.485445
Minibatch accuracy: 75.0%
Validation accuracy: 88.8%
Minibatch loss at step 5100: 0.235095
Minibatch accuracy: 87.

Minibatch loss at step 9100: 0.307969
Minibatch accuracy: 87.5%
Validation accuracy: 89.6%
Minibatch loss at step 9150: 0.540278
Minibatch accuracy: 87.5%
Validation accuracy: 89.6%
Minibatch loss at step 9200: 0.240823
Minibatch accuracy: 93.8%
Validation accuracy: 90.0%
Minibatch loss at step 9250: 0.169041
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 9300: 0.307019
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 9350: 0.191416
Minibatch accuracy: 93.8%
Validation accuracy: 90.1%
Minibatch loss at step 9400: 0.422033
Minibatch accuracy: 87.5%
Validation accuracy: 89.7%
Minibatch loss at step 9450: 0.663401
Minibatch accuracy: 81.2%
Validation accuracy: 90.0%
Minibatch loss at step 9500: 0.630686
Minibatch accuracy: 75.0%
Validation accuracy: 89.8%
Minibatch loss at step 9550: 0.505710
Minibatch accuracy: 81.2%
Validation accuracy: 90.0%
Minibatch loss at step 9600: 0.739411
Minibatch accuracy: 81.2%
Validation accuracy: 89.8%

Minibatch loss at step 13600: 0.542433
Minibatch accuracy: 87.5%
Validation accuracy: 90.6%
Minibatch loss at step 13650: 0.026442
Minibatch accuracy: 100.0%
Validation accuracy: 90.0%
Minibatch loss at step 13700: 0.160212
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 13750: 0.057764
Minibatch accuracy: 100.0%
Validation accuracy: 90.3%
Minibatch loss at step 13800: 0.275685
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 13850: 0.177976
Minibatch accuracy: 93.8%
Validation accuracy: 90.4%
Minibatch loss at step 13900: 0.579023
Minibatch accuracy: 81.2%
Validation accuracy: 90.5%
Minibatch loss at step 13950: 0.225546
Minibatch accuracy: 93.8%
Validation accuracy: 90.5%
Minibatch loss at step 14000: 0.680975
Minibatch accuracy: 87.5%
Validation accuracy: 90.4%
Minibatch loss at step 14050: 0.004413
Minibatch accuracy: 100.0%
Validation accuracy: 90.6%
Minibatch loss at step 14100: 0.092585
Minibatch accuracy: 100.0%
Validation 

Minibatch loss at step 18100: 0.249996
Minibatch accuracy: 87.5%
Validation accuracy: 90.8%
Minibatch loss at step 18150: 0.245187
Minibatch accuracy: 93.8%
Validation accuracy: 90.7%
Minibatch loss at step 18200: 0.167561
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 18250: 0.152560
Minibatch accuracy: 93.8%
Validation accuracy: 90.4%
Minibatch loss at step 18300: 0.533938
Minibatch accuracy: 87.5%
Validation accuracy: 90.8%
Minibatch loss at step 18350: 0.084572
Minibatch accuracy: 100.0%
Validation accuracy: 90.8%
Minibatch loss at step 18400: 0.084313
Minibatch accuracy: 93.8%
Validation accuracy: 90.8%
Minibatch loss at step 18450: 0.215625
Minibatch accuracy: 93.8%
Validation accuracy: 90.8%
Minibatch loss at step 18500: 0.128858
Minibatch accuracy: 93.8%
Validation accuracy: 90.7%
Minibatch loss at step 18550: 0.372032
Minibatch accuracy: 75.0%
Validation accuracy: 90.9%
Minibatch loss at step 18600: 0.012544
Minibatch accuracy: 100.0%
Validation ac

Minibatch loss at step 22600: 0.245044
Minibatch accuracy: 93.8%
Validation accuracy: 90.9%
Minibatch loss at step 22650: 0.086303
Minibatch accuracy: 100.0%
Validation accuracy: 90.9%
Minibatch loss at step 22700: 0.156807
Minibatch accuracy: 93.8%
Validation accuracy: 91.1%
Minibatch loss at step 22750: 0.018767
Minibatch accuracy: 100.0%
Validation accuracy: 91.2%
Minibatch loss at step 22800: 0.183960
Minibatch accuracy: 87.5%
Validation accuracy: 91.0%
Minibatch loss at step 22850: 0.004873
Minibatch accuracy: 100.0%
Validation accuracy: 90.9%
Minibatch loss at step 22900: 0.140222
Minibatch accuracy: 100.0%
Validation accuracy: 90.9%
Minibatch loss at step 22950: 0.245438
Minibatch accuracy: 93.8%
Validation accuracy: 90.9%
Minibatch loss at step 23000: 0.388667
Minibatch accuracy: 87.5%
Validation accuracy: 91.1%
Minibatch loss at step 23050: 0.318889
Minibatch accuracy: 93.8%
Validation accuracy: 90.9%
Minibatch loss at step 23100: 0.265233
Minibatch accuracy: 87.5%
Validation 

Validation accuracy: 91.2%
Minibatch loss at step 27100: 0.382216
Minibatch accuracy: 87.5%
Validation accuracy: 90.8%
Minibatch loss at step 27150: 0.020263
Minibatch accuracy: 100.0%
Validation accuracy: 91.4%
Minibatch loss at step 27200: 0.237335
Minibatch accuracy: 93.8%
Validation accuracy: 91.2%
Minibatch loss at step 27250: 0.003363
Minibatch accuracy: 100.0%
Validation accuracy: 91.2%
Minibatch loss at step 27300: 0.299435
Minibatch accuracy: 87.5%
Validation accuracy: 91.5%
Minibatch loss at step 27350: 0.529766
Minibatch accuracy: 81.2%
Validation accuracy: 91.5%
Minibatch loss at step 27400: 0.425509
Minibatch accuracy: 81.2%
Validation accuracy: 91.3%
Minibatch loss at step 27450: 0.187088
Minibatch accuracy: 93.8%
Validation accuracy: 91.3%
Minibatch loss at step 27500: 0.188439
Minibatch accuracy: 93.8%
Validation accuracy: 91.3%
Minibatch loss at step 27550: 0.574399
Minibatch accuracy: 68.8%
Validation accuracy: 91.2%
Minibatch loss at step 27600: 0.364539
Minibatch ac

numhidden=128 
Validation accuracy: 91.4%
Test accuracy: 96.4%
numhidden=256
Validation accuracy: 91.6%
Test accuracy: 96.6%